In [ ]:
!pip install river
!pip install pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import river
import datetime
import json
import datetime
import pandas as pd
import tokenize
import pprint

from river import preprocessing
from river import linear_model
from river import optim
from river import metrics
from river import evaluate

from river import feature_extraction

from river import datasets
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import compose


In [ ]:
# PANDAS-FRIENDLY
read_start = datetime.datetime.now()

with open('combined_names_newest.json') as f:
    dataset = json.load(f)

# NB: this is getting all used pt_ids, also reformatting in something more like what we might use
pts = set()
for item in dataset:
    item_pts = {}
    # NB: this replaces the provision_type_ids list;
    # the file has it as a list of strings like this: "label-true/false"
    # we reformat the list so it is a list of tuples: ['label', bool]
    for label in item['labels']['provision_type_ids']:
        l = label.rsplit('-', 1)
        if l[1] == 'true':
            l[1] = True
        else:
            l[1] = False
        pts.add(l[0])
        item_pts[l[0]] = l[1]
        # if l[0] == 'Governing Law':
        #     pts.add(l[0])
        #     item_pts.append(l)
    item['labels']['provision_type_ids'] = item_pts


read_end = datetime.datetime.now()
print('file load:', (read_end - read_start))

print(dataset[0])

file load: 0:00:02.090676
{'id': 's6avtdg0pl_QbjGMOcU1', 'features': {'core_lemma_text': 'i Delivery amount return amount credit support amount', 'compounded_core_lemma_text': 'i Deliveryamount returnamount creditsupportamount'}, 'labels': {'provision_type_ids': {'CSA Delivery Amount, Return Amount, and Credit Support Amount': True, 'CSA Credit Support Obligations': True, 'CSA Paragraph 11/13': True, 'CSA Eligible Credit Support / Collateral': False, 'CSA Resolution Time': False, 'CSA Dispute Resolution': False, 'CSA Demands and Notices': False, 'CSA Other Provisions': False, 'CSA Valuation Time': False, 'CSA Valuation and Timing': False, 'CSA Exchange Date': False, 'Addresses for Transfers': False, 'CSA Threshold': False, 'CSA Thresholds': False, 'CSA Minimum Transfer Amount': False, 'CSA Credit Support Amount': False, 'CSA Value': False, 'CSA Return Amount': False, 'CSA Base Currency and Eligible Currency': False, 'CSA Delivery Amount': False, 'CSA Valuation Agent': False, 'CSA Valua

In [ ]:
test_dataset = dataset[-10:]

normalized = pd.json_normalize(test_dataset)
#pprint.pprint(normalized)
# # create a list of strings
#columns = ['id', 'features.core_lemma_text', 'labels.provision_type_ids["CSA Delivery Amount, Return Amount, and Credit Support Amount"]']


# # Passing a dictionary
# # key: column name
# # value: series of values
df = pd.DataFrame(normalized)
#print(df)
#new_series = df.squeeze()
#print(new_series)
#print(df)
# ser = pd.Series(df['features.core_lemma_text'],index=df["id"])
# pprint.pprint(ser)

In [ ]:
df

,id,features.core_lemma_text,features.compounded_core_lemma_text,"labels.provision_type_ids.CSA Delivery Amount, Return Amount, and Credit Support Amount",labels.provision_type_ids.CSA Credit Support Obligations,labels.provision_type_ids.CSA Paragraph 11/13,labels.provision_type_ids.CSA Eligible Credit Support / Collateral,labels.provision_type_ids.CSA Resolution Time,labels.provision_type_ids.CSA Dispute Resolution,labels.provision_type_ids.CSA Demands and Notices,...,labels.provision_type_ids.ISDA Additional Representation,labels.provision_type_ids.ISDA Multibranch Party,labels.provision_type_ids.ISDA Specified Transaction,labels.provision_type_ids.ISDA Termination Currency,labels.provision_type_ids.Recording of Conversations,labels.provision_type_ids.Relationship Between Parties,labels.provision_type_ids.ISDA Absence of Litigation,labels.provision_type_ids.Failure to Pay or Deliver,labels.provision_type_ids.Scope of Agreement,labels.provision_type_ids.Export of Defaults
0,v2Oh0IETww3WAd2-tiwF,New York NY 10022,NewYork NY 10022,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,q36Vd5XFPTk2S8gHHPoi,attention back office,attention backoffice,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,j8E7ZSgG7qqEvjIrNHk_,telex no. 649462,telexno. 649462,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,p9TpwUaWXTvrqd1y-vHG,facsimile 45 33 91 1521,facsimile 45 33 91 1521,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,r9T4zjsW5Rcw3OttNaVb,attn. management,attn. management,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,f8oRo6v1THrrT7WBhMSA,SW1X 9SR,SW1X 9SR,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,y1h5Av4u-dqNs5Ijd9BA,c Offices provision section 10a apply agreement,c Offices provision section 10a apply agreement,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,o4LyYnXaqwb1UMC8Fb4c,follow provision insert section 6f,follow provision insert section 6f,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,v2-IBGNEmOqs6NNyIbxU,4 amendment,4 amendment,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,d1HbEkbM6zwPUB6mhW0G,5 Indemnifiable tax,5 Indemnifiabletax,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [46]:
tfidf = feature_extraction.TFIDF()

sentance = df['features.core_lemma_text']
tf = []
for s in sentance:
    tfidf = tfidf.learn_one(s)
    print(tfidf.transform_one(s))
    tf.append(tfidf.transform_one(s))

{'new': 0.5, 'york': 0.5, 'ny': 0.5, '10022': 0.5}
{'attention': 0.5773502691896258, 'back': 0.5773502691896258, 'office': 0.5773502691896258}
{'telex': 0.5773502691896257, 'no': 0.5773502691896257, '649462': 0.5773502691896257}
{'facsimile': 0.4472135954999579, '45': 0.4472135954999579, '33': 0.4472135954999579, '91': 0.4472135954999579, '1521': 0.4472135954999579}
{'attn': 0.7071067811865475, 'management': 0.7071067811865475}
{'sw1x': 0.7071067811865476, '9sr': 0.7071067811865476}
{'offices': 0.408248290463863, 'provision': 0.408248290463863, 'section': 0.408248290463863, '10a': 0.408248290463863, 'apply': 0.408248290463863, 'agreement': 0.408248290463863}
{'follow': 0.47647419687686043, 'provision': 0.399322563306691, 'insert': 0.47647419687686043, 'section': 0.399322563306691, '6f': 0.47647419687686043}
{'amendment': 1.0}
{'indemnifiable': 0.7071067811865475, 'tax': 0.7071067811865475}


In [47]:
tf

[{'10022': 0.5, 'new': 0.5, 'ny': 0.5, 'york': 0.5},
 {'attention': 0.5773502691896258,
  'back': 0.5773502691896258,
  'office': 0.5773502691896258},
 {'649462': 0.5773502691896257,
  'no': 0.5773502691896257,
  'telex': 0.5773502691896257},
 {'1521': 0.4472135954999579,
  '33': 0.4472135954999579,
  '45': 0.4472135954999579,
  '91': 0.4472135954999579,
  'facsimile': 0.4472135954999579},
 {'attn': 0.7071067811865475, 'management': 0.7071067811865475},
 {'9sr': 0.7071067811865476, 'sw1x': 0.7071067811865476},
 {'10a': 0.408248290463863,
  'agreement': 0.408248290463863,
  'apply': 0.408248290463863,
  'offices': 0.408248290463863,
  'provision': 0.408248290463863,
  'section': 0.408248290463863},
 {'6f': 0.47647419687686043,
  'follow': 0.47647419687686043,
  'insert': 0.47647419687686043,
  'provision': 0.399322563306691,
  'section': 0.399322563306691},
 {'amendment': 1.0},
 {'indemnifiable': 0.7071067811865475, 'tax': 0.7071067811865475}]

In [30]:
model = compose.Pipeline(
     preprocessing.StandardScaler(),
     linear_model.LogisticRegression()
 )

In [31]:
metric = metrics.Accuracy()

In [ ]:
columns = list(df)

del columns[:3]


print(columns)
len(columns)

rows = len(df.index.values)

['labels.provision_type_ids.CSA Delivery Amount, Return Amount, and Credit Support Amount', 'labels.provision_type_ids.CSA Credit Support Obligations', 'labels.provision_type_ids.CSA Paragraph 11/13', 'labels.provision_type_ids.CSA Eligible Credit Support / Collateral', 'labels.provision_type_ids.CSA Resolution Time', 'labels.provision_type_ids.CSA Dispute Resolution', 'labels.provision_type_ids.CSA Demands and Notices', 'labels.provision_type_ids.CSA Other Provisions', 'labels.provision_type_ids.CSA Valuation Time', 'labels.provision_type_ids.CSA Valuation and Timing', 'labels.provision_type_ids.CSA Exchange Date', 'labels.provision_type_ids.Addresses for Transfers', 'labels.provision_type_ids.CSA Threshold', 'labels.provision_type_ids.CSA Thresholds', 'labels.provision_type_ids.CSA Minimum Transfer Amount', 'labels.provision_type_ids.CSA Credit Support Amount', 'labels.provision_type_ids.CSA Value', 'labels.provision_type_ids.CSA Return Amount', 'labels.provision_type_ids.CSA Base Cu

In [ ]:
metric = metrics.Accuracy()


In [ ]:
from sklearn.preprocessing import LabelEncoder

y_encoded = []
le= LabelEncoder()
for i in columns:
  y_encoded.append(le.fit_transform(df[i]))  






In [23]:
print(y_encoded)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0,

In [49]:
for i in tf:
  y_pred = model.predict_one(i)

In [42]:
for i in df['labels.provision_type_ids.Address for Notices']:
  metric = metric.update(i, y_pred)

In [43]:
for i in df['labels.provision_type_ids.Address for Notices']:
  model = model.learn_one(a, i)